In [2]:
## Chain 1 : 기본 모델
from langchain_openai import ChatOpenAI

chain = ChatOpenAI() 
chain.invoke("안녕!")

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 12, 'total_tokens': 33, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9b2dfffe-5ae3-466c-bf1e-cc726722757e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [3]:
## Chain 2 : 모델 + 출력
from langchain.schema.output_parser import StrOutputParser

chain = ChatOpenAI() | StrOutputParser()
chain.invoke("안녕!")

'안녕하세요! 무엇을 도와드릴까요?'

In [4]:
# Chain 3 : 프롬프트 → 모델 → 출력
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ('system', "다음의 언어로 번역하세요 {language}:"),
    ('user', '{text}')
])

chain = prompt_template | ChatOpenAI() | StrOutputParser()
chain.invoke({"language": "영어", "text": "안녕"})

'Hello'

In [5]:
# Chain 4 : 질문 → 웹문서 → 유사텍스트

## 사전 준비
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings


# 1. 블로그 내용을 불러오고 청킹, 벡터DB에 저장
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)


retriever = vectorstore.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = retriever | format_docs
chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\n\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority

In [7]:
# Chain 5 : 질문 → 웹문서 → 유사텍스트 -> 모델응답 (RAG)
from langchain_core.runnables import RunnablePassthrough ## 체인에서 데이터를 전달하는 역할
from langchain_core.prompts import PromptTemplate


template = """
다음의 컨텍스트를 활용해서 질문에 답변해줘
- 질문에 대한 응답을 해줘
- 간결하게 5줄 이내로 해줘
- 곧바로 응답결과를 말해줘

컨텍스트 : {context}

질문: {question}

응답:"""

rag_prompt = PromptTemplate.from_template(template)
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt | ChatOpenAI() | StrOutputParser()
)

qa_chain.invoke("너의 이름은 뭐니?")


'제 이름은 MRKL (Karpas et al. 2022) 입니다.'

In [8]:
## RunnablePassthrough 안쓰기

def get_qa_chain_answer(question):
    retrieve_docs = retriever.invoke(question)
    formatted_docs = format_docs(retrieve_docs)

    ## RAG 체인 선언
    chain = rag_prompt | ChatOpenAI() | StrOutputParser()
    response = chain.invoke({"question": question, "context": formatted_docs})

    return response

print(get_qa_chain_answer("너의 이름은 워니"))

저의 이름은 워니가 아닙니다.
